In [2]:
import numpy as np
import pandas as pd
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [4]:
data = pd.read_csv('../../deepmoji/data/train.txt', sep = '\t')

In [5]:
label2emotion = {0: "others", 1: "happy", 2: "sad", 3: "angry"}
emotion2label = {"others": 0, "happy": 1, "sad": 2, "angry": 3}

emoticons_additional = {
    '(^・^)': '<happy>', ':‑c': '<sad>', '=‑d': '<happy>', ":'‑)": '<happy>', ':‑d': '<laugh>',
    ':‑(': '<sad>', ';‑)': '<happy>', ':‑)': '<happy>', ':\\/': '<sad>', 'd=<': '<annoyed>',
    ':‑/': '<annoyed>', ';‑]': '<happy>', '(^�^)': '<happy>', 'angru': 'angry', "d‑':":
        '<annoyed>', ":'‑(": '<sad>', ":‑[": '<annoyed>', '(�?�)': '<happy>', 'x‑d': '<laugh>',
}

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
               'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
              'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter",
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter",
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons, emoticons_additional]
)


def tokenize(text):
    text = " ".join(text_processor.pre_process_doc(text))
    return text

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [6]:
x1=[]
x2=[]
x3=[]
y=[]
labels={}
i=0
for ind, row in data.iterrows():
    if row['label'] not in labels:
        labels[row['label']]=i
        i+=1
    y.append(labels[row['label']])
    x1.append(tokenize(row['turn1']))
    x2.append(tokenize(row['turn2']))
    x3.append(tokenize(row['turn3']))

In [9]:
x3

["what ' s ur name ?",
 'no . i never saw you',
 'where you live',
 'u little disgusting whore',
 'maybe',
 'that <elongated> why',
 'what are you watching on tv ?',
 'so , how are u',
 'y saying so many times . <repeated> i can hear you',
 '😘 love you',
 'yes',
 'not giving whatsapp no .',
 'love you too',
 'i have no gf',
 'ok',
 '😁 😁',
 'me *',
 'i do not want to talk u any more',
 '😭 😭',
 '😑',
 'what *',
 '😂 😂 😂 so you have legs too',
 'even i do not',
 'do u know what <number> is ?',
 '😢',
 'which is your favourite movie',
 'im really sad today',
 'first you send your email address received than send first young lady',
 'india',
 'you know you are a machine , right ?',
 'u didnt ans my question',
 'wlcm',
 'haha . <repeated> this is evening',
 'you are dumb',
 'i fuck u . <repeated> open ur panty',
 'i want to propose a girl',
 'you explain me that',
 'no i never go for there',
 'are you bored of me ?',
 'haha yes ofcorse <wink>',
 'what',
 'let us start',
 'very nice friends',
 '

In [14]:
from __future__ import print_function, division, unicode_literals
import json

from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_feature_encoding
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH

def encode_deepmoji(x):
    maxlen = 30
    batch_size = 32

    print('Tokenizing using dictionary from {}'.format(VOCAB_PATH))
    with open(VOCAB_PATH, 'r') as f:
        vocabulary = json.load(f)
    st = SentenceTokenizer(vocabulary, maxlen)
    tokenized, _, _ = st.tokenize_sentences(x)
    print('Loading model from {}.'.format(PRETRAINED_PATH))
    model = torchmoji_feature_encoding(PRETRAINED_PATH)
    print(model)
    print('Encoding texts..')
    encoding = np.zeros((len(x), 2304))
    for i in range(0, len(x), 300):    
        encoding[i:i+300] = model(tokenized[i:i+300])
    return encoding

In [ ]:
x1_vec = encode_deepmoji(x1)
np.save('deepmoji_train_x1.npy', x1_vec)

In [15]:
x2_vec = encode_deepmoji(x2)

Tokenizing using dictionary from /home/Jay/Notebooks/deepmoji/torchMoji/model/vocabulary.json
Loading model from /home/Jay/Notebooks/deepmoji/torchMoji/model/pytorch_model.bin.
Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0.bias_ih_l0_reverse
Loading weights for lstm_0.bias_hh_l0_reverse
Loading weights for lstm_1.weight_ih_l0
Loading weights for lstm_1.weight_hh_l0
Loading weights for lstm_1.bias_ih_l0
Loading weights for lstm_1.bias_hh_l0
Loading weights for lstm_1.weight_ih_l0_reverse
Loading weights for lstm_1.weight_hh_l0_reverse
Loading weights for lstm_1.bias_ih_l0_reverse
Loading weights for lstm_1.bias_hh_l0_reverse
Loading weights for attention_layer.attention_vector
Ignoring weights for output_layer.0.weight
Ignori

In [23]:
np.savez('deepmoji_train_x2.npy.z', x2_vec)

In [19]:
x3_vec = encode_deepmoji(x3)
np.savez('deepmoji_train_x3.npy.z', x3_vec)

Tokenizing using dictionary from /home/Jay/Notebooks/deepmoji/torchMoji/model/vocabulary.json
Loading model from /home/Jay/Notebooks/deepmoji/torchMoji/model/pytorch_model.bin.
Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0.bias_ih_l0_reverse
Loading weights for lstm_0.bias_hh_l0_reverse
Loading weights for lstm_1.weight_ih_l0
Loading weights for lstm_1.weight_hh_l0
Loading weights for lstm_1.bias_ih_l0
Loading weights for lstm_1.bias_hh_l0
Loading weights for lstm_1.weight_ih_l0_reverse
Loading weights for lstm_1.weight_hh_l0_reverse
Loading weights for lstm_1.bias_ih_l0_reverse
Loading weights for lstm_1.bias_hh_l0_reverse
Loading weights for attention_layer.attention_vector
Ignoring weights for output_layer.0.weight
Ignori

In [20]:
def print_embedding(i):
    print('First 5 dimensions for sentence: {}'.format(x1[i]))
    print(deepmoji_train[i])

In [22]:
x2_vec == x3_vec

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [35]:
import sklearn as sk 
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=0).fit(deepmoji_train, y) 

In [36]:
LR.predict(deepmoji_train[27000:])  
round(LR.score(deepmoji_train[27000:],y[27000:]), 4)

0.63919999999999999